### TO DO ###

- Fix no data for first week
- Check by week when new data comes
- Add value for players who don't play
- Goal difference statistic
- Above or below average score
- Pandas Stylesheet
- Graphs

In [1]:
import requests
import pandas as pd
import numpy as np

#### Definitions ####

In [2]:
owners = ['Luke','Emma','Fat','Olivia','Claire','Matt','George']
owner_ids = ['18900','18935','18967','19180','19424','20043','20626']
team_details_url = {}
lineup_url = {}

for owner_id, owner in enumerate(owners):
    team_details_url[owner] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/history'
    for gameweek in range(1,39):
        lineup_url[owner,gameweek] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/event/'+str(gameweek)

bootstrap_url = 'https://draft.premierleague.com/api/bootstrap-static'
fantasy_bootstrap_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
player_owner_url = 'https://draft.premierleague.com/api/league/5857/element-status'
league_details_url = 'https://draft.premierleague.com/api/league/5857/details'

current_gameweek = requests.get(bootstrap_url).json()['events']['current']
total_fantasy_players = requests.get(fantasy_bootstrap_url).json()['total_players']

In [3]:
def url_to_df(url, datatype):
    request = requests.get(url)
    json = request.json()
    df = pd.DataFrame(json[datatype])
    return df

#### Create Dataframes ####

In [4]:
player_types_df = url_to_df(bootstrap_url,'element_types')

teams_df = url_to_df(bootstrap_url,'teams')

player_owner_df = url_to_df(player_owner_url,'element_status')
player_owner_df = player_owner_df.sort_values('element', ignore_index=True)

league_entries_df = url_to_df(league_details_url,'league_entries')
league_standings_df = url_to_df(league_details_url,'standings')
league_standings_df = league_standings_df.sort_values('league_entry', ignore_index=True)
league_standings_df = pd.concat([league_entries_df[['entry_id','id','entry_name','player_first_name','waiver_pick']], league_standings_df[['last_rank','rank','event_total','total']]], axis=1)

player_info_df = url_to_df(bootstrap_url,'elements')
player_info_df['team'] = player_info_df.team.map(teams_df.set_index('id').name)
player_info_df['element_type'] = player_info_df.element_type.map(player_types_df.set_index('id').singular_name)
player_info_df.insert(3,'owner',player_owner_df.owner.map(league_entries_df.set_index('entry_id').player_first_name))

#### Functions for making specific DataFrames ####

In [13]:
def get_lineup_info(owner, gameweek = current_gameweek):
    # check valid gameweek:
    if gameweek > current_gameweek:
        print("Gameweek not yet played")
        return('ERROR')
    
    # create dataframe:
    info_columns = ['name','team','position']
    points_columns = ['minutes','goals_scored','assists','saves','goals_conceded',
                      'own_goals','yellow_cards','red_cards','total_points']
    lineup_df = pd.DataFrame(columns=info_columns + ['value', 'selected %'] + points_columns,
                             index = url_to_df(lineup_url[owner, gameweek],'picks')['element'])
    
    # all player data:
    lineup_df['name'] = lineup_df.index.map(player_info_df.set_index('id').second_name)
    lineup_df['team'] = lineup_df.index.map(player_info_df.set_index('id').team)
    lineup_df['position'] = lineup_df.index.map(player_info_df.set_index('id').element_type)
    
    # player by player data:
    for i, player in enumerate(lineup_df.index):
        points_collection = {}
        for data in points_columns:
            points_collection[data] = 0
        player_url = 'https://draft.premierleague.com/api/element-summary/'+str(player)
        player_df = url_to_df(player_url,'history')
        fantasy_player_url = 'https://fantasy.premierleague.com/api/element-summary/'+str(player)+'/'
        fantasy_player_df = url_to_df(fantasy_player_url,'history')
        if not player_df.empty:
            lineup_df.loc[lineup_df.index == player, 'value'] = fantasy_player_df.loc[fantasy_player_df['round'] == gameweek]['value'].iloc[0]/10
            lineup_df.loc[lineup_df.index == player, 'selected %'] = fantasy_player_df.loc[fantasy_player_df['round'] == gameweek]['selected'].iloc[0]/total_fantasy_players*100
            player_df = player_df.loc[player_df['event'] == gameweek]
            if not player_df.empty:
                for data in points_columns:
                    points_collection[data] = player_df[data].sum()
        for data in points_columns:
            lineup_df.loc[lineup_df.index == player, data] = points_collection[data]
            
    # add totals row:
    totals_df = pd.DataFrame(index = ['Totals (inc. bench)','Totals'])
    for data in info_columns:
        totals_df.loc[:, data] = ''
    for data in ['value'] + points_columns:
        totals_df.loc['Totals (inc. bench)', data] = lineup_df[data].sum()
        totals_df.loc['Totals', data] = lineup_df.head(11)[data].sum()
    totals_df.loc['Totals (inc. bench)', 'name'] = sum(lineup_df['minutes']>0)
    totals_df.loc['Totals', 'name'] = sum(lineup_df.head(11)['minutes']>0)
    totals_df.loc['Totals (inc. bench)', 'selected %'] = lineup_df['selected %'].mean()
    totals_df.loc['Totals', 'selected %'] = lineup_df.head(11)['selected %'].mean()
    totals_df.loc['Totals (inc. bench)', 'team'] = lineup_df['team'].nunique()
    totals_df.loc['Totals', 'team'] = lineup_df.head(11)['team'].nunique()
    lineup_df = pd.concat([lineup_df, totals_df])
        
    # make reader friendly:
    lineup_df.index.name = None
    lineup_df = lineup_df.rename(columns={'goals_scored': 'goals scored', 'goals_conceded': 'goals conceded',
                                          'own_goals': 'own goals', 'yellow_cards': 'yellow cards',
                                          'red_cards': 'red cards', 'total_points': 'total points'})
    return(lineup_df)

In [14]:
def get_league_standings(Cumulative=True):
    gameweek_range = range(1,current_gameweek+1)
    # create dataframe:
    league_standings_df = pd.DataFrame(columns = gameweek_range, index = owners)
    team_info_df = {}
    for owner in owners:
        team_info_df[owner] = url_to_df(team_details_url[owner],'history')
        for week in gameweek_range:
            points = team_info_df[owner].loc[team_info_df[owner]['event'] == week,'points'].iloc[0]
            league_standings_df.loc[owner, week] = points
        league_standings_df.loc[owner, 'Totals'] = team_info_df[owner].sum()['points']
        
    return(league_standings_df)

In [15]:
get_league_standings(True)

,1,2,Totals
Luke,27,0,27.0
Emma,43,0,43.0
Fat,48,0,48.0
Olivia,31,0,31.0
Claire,39,0,39.0
Matt,22,0,22.0
George,26,0,26.0


#### Get all info (only run when new info to retrieve) ####

In [17]:
def get_all_lineups():
    all_lineups_df = {}
    for gameweek in range(1, current_gameweek+1):
        for owner in owners:
            print(owner, gameweek)
            all_lineups_df[owner, gameweek] = get_lineup_info(owner, gameweek)
    return(all_lineups_df)

In [18]:
all_lineups_df = get_all_lineups()

Luke 1
Emma 1
Fat 1
Olivia 1
Claire 1
Matt 1
George 1
Luke 2
Emma 2
Fat 2
Olivia 2
Claire 2
Matt 2
George 2


#### Make all info readable ####

In [19]:
def get_league_stats_df_by_gameweek(all_lineups_df, gameweek, include_bench = False):
    if include_bench:
        value_to_get = 'Totals (inc. bench)'
    else:
        value_to_get = 'Totals'
    stats = ['name','team','value','selected %','minutes','goals scored','assists','saves','goals conceded','own goals','yellow cards','red cards','total points']
    league_stats_df = pd.DataFrame(columns = stats + ['position'], index = owners)
    league_stats_df.insert(5, 'productivity', 0)
    
    for owner in owners:
        for stat in stats:
            league_stats_df.loc[owner, stat] = all_lineups_df[owner, gameweek].loc[value_to_get,stat]
    league_stats_df = league_stats_df.sort_values('total points', ascending=False)
    for i, owner in enumerate(league_stats_df.index):
        league_stats_df.loc[owner, 'productivity'] = league_stats_df.loc[owner, 'total points']/league_stats_df.loc[owner, 'minutes']
        league_stats_df.loc[owner, 'position'] = (i+1)
    league_stats_df = league_stats_df.rename(columns={'name': 'active players', 'total points': 'points'})
    return(league_stats_df)

In [20]:
def get_league_stats_df_by_stat(all_lineups_df, stat, include_bench = False):
    if include_bench:
        value_to_get = 'Totals (inc. bench)'
    else:
        value_to_get = 'Totals'
    gameweek_range = range(1,current_gameweek+1)
    league_stats_df = pd.DataFrame(columns = gameweek_range, index = owners)
    
    for owner in owners:
        for gameweek in gameweek_range:
            league_stats_df.loc[owner, gameweek] = all_lineups_df[owner, gameweek].loc[value_to_get,stat]
    return(league_stats_df)

In [23]:
stats_df = get_league_stats_df_by_gameweek(all_lineups_df, 1, False)
stats_df

,active players,team,value,selected %,minutes,productivity,goals scored,assists,saves,goals conceded,own goals,yellow cards,red cards,points,position
Fat,11,8,0,NaN,913,0.052574,3,1,3,11,0,0,0,48,1
Emma,10,9,0,NaN,786,0.054707,3,0,0,14,0,0,0,43,2
Claire,10,9,0,NaN,799,0.048811,2,3,0,13,0,2,0,39,3
Olivia,11,8,0,NaN,953,0.032529,2,1,2,13,0,2,0,31,4
Luke,9,8,0,NaN,610,0.044262,0,1,1,7,0,0,0,27,5
George,9,9,0,NaN,685,0.037956,0,0,5,10,0,1,0,26,6
Matt,7,9,0,NaN,509,0.043222,2,1,0,12,0,0,0,22,7


In [303]:
stat_to_sort_by = 'goals conceded'
stats_df.sort_values(stat_to_sort_by, ascending=False)[[stat_to_sort_by,'points','position']]

,goals conceded,points,position
Emma,14,43,2
Claire,13,39,3
Olivia,13,31,4
Matt,12,22,7
Fat,11,48,1
George,10,26,6
Luke,7,27,5


In [26]:
url_to_df('https://fantasy.premierleague.com/api/element-summary/128/','history')

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,128,1,16,10,True,2020-09-12T14:00:00Z,1.0,0.0,1,90,...,31,41.4,0.0,0.0,4.1,50,0,87269,0,0
1,128,15,13,0,False,2020-09-19T16:30:00Z,NaN,NaN,2,0,...,0,0.0,0.0,0.0,0.0,50,30722,203846,49711,18989
